# Torch version inspection

In [1]:
from torch.utils import model_zoo
import torch
from torchvision import models

In [2]:
RESTORE_FROM = 'http://vllab1.ucmerced.edu/~whung/adv-semi-seg/resnet101COCO-41f33a49.pth'

In [3]:
def load_pretrained_weights(restore_from, model):
    # TODO: convert to Tensorflow...? load pretrained parameters
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    if restore_from.startswith('http'):
        saved_state_dict = model_zoo.load_url(restore_from, map_location=device)
    else:
        saved_state_dict = torch.load(restore_from, map_location=torch.device('cpu') )

    # only copy the params that exist in current model (caffe-like)
    new_params = model.state_dict().copy()
    for name, param in new_params.items():
        print(name)
        if name in saved_state_dict and param.size() == saved_state_dict[name].size():
            new_params[name].copy_(saved_state_dict[name])
            print('copy {}'.format(name))
    model.load_state_dict(new_params)

In [4]:
model = models.resnet18(pretrained=False)  
load_pretrained_weights(RESTORE_FROM, model)


/opt/anaconda3/envs/csci1470/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/csci1470/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


conv1.weight
copy conv1.weight
bn1.weight
copy bn1.weight
bn1.bias
copy bn1.bias
bn1.running_mean
copy bn1.running_mean
bn1.running_var
copy bn1.running_var
bn1.num_batches_tracked
layer1.0.conv1.weight
layer1.0.bn1.weight
copy layer1.0.bn1.weight
layer1.0.bn1.bias
copy layer1.0.bn1.bias
layer1.0.bn1.running_mean
copy layer1.0.bn1.running_mean
layer1.0.bn1.running_var
copy layer1.0.bn1.running_var
layer1.0.bn1.num_batches_tracked
layer1.0.conv2.weight
copy layer1.0.conv2.weight
layer1.0.bn2.weight
copy layer1.0.bn2.weight
layer1.0.bn2.bias
copy layer1.0.bn2.bias
layer1.0.bn2.running_mean
copy layer1.0.bn2.running_mean
layer1.0.bn2.running_var
copy layer1.0.bn2.running_var
layer1.0.bn2.num_batches_tracked
layer1.1.conv1.weight
layer1.1.bn1.weight
copy layer1.1.bn1.weight
layer1.1.bn1.bias
copy layer1.1.bn1.bias
layer1.1.bn1.running_mean
copy layer1.1.bn1.running_mean
layer1.1.bn1.running_var
copy layer1.1.bn1.running_var
layer1.1.bn1.num_batches_tracked
layer1.1.conv2.weight
copy layer1

# Example loading pretrained parameters usage

In [1]:
import tensorflow as tf
checkpoint_path = '../model/pretrained/Deeplab Resnet.ckpt'

def restore_model_from_checkpoint(checkpoint_path, model):
    reader = tf.train.load_checkpoint(checkpoint_path)
    restore_dict = reader.get_variable_to_shape_map()

    for var in model.variables:
       if var.path in restore_dict:
            tensor = reader.get_tensor(var.path)
            var.assign(tensor)
            print(f'Successfully loaded {var.path} into the model.')
            
    print("Model restored with subset matching from:", checkpoint_path)



In [2]:
def build_custom_model():
    inputs = tf.keras.Input(shape=(224, 224, 3))  # Example input shape for an image

    # Initial layers
    x = tf.keras.layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='conv1')(inputs)
    x = tf.keras.layers.BatchNormalization(name='bn_conv1')(x)
    x = tf.keras.layers.Activation('relu')(x)
    x = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Layer with controlled variable name
    x = tf.keras.layers.Conv2D(256, (3, 3), padding='same', name='res2c_branch2c')(x)
    x = tf.keras.layers.BatchNormalization(name='bn2c_branch2c')(x)
    x = tf.keras.layers.Activation('relu')(x)

    # Output layer
    outputs = tf.keras.layers.Dense(1000, activation='softmax')(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

# Build and check model variables
model = build_custom_model()
for v in model.variables:
    print(v.path)
print(model.variables[:3])

conv1/kernel
conv1/bias
bn_conv1/gamma
bn_conv1/beta
bn_conv1/moving_mean
bn_conv1/moving_variance
res2c_branch2c/kernel
res2c_branch2c/bias
bn2c_branch2c/gamma
bn2c_branch2c/beta
bn2c_branch2c/moving_mean
bn2c_branch2c/moving_variance
dense/kernel
dense/bias
[<KerasVariable shape=(7, 7, 3, 64), dtype=float32, path=conv1/kernel>, <KerasVariable shape=(64,), dtype=float32, path=conv1/bias>, <KerasVariable shape=(64,), dtype=float32, path=bn_conv1/gamma>]


In [4]:
restore_model_from_checkpoint(checkpoint_path, model)

Successfully loaded bn_conv1/gamma into the model.
Successfully loaded bn_conv1/beta into the model.
Successfully loaded bn_conv1/moving_mean into the model.
Successfully loaded bn_conv1/moving_variance into the model.
Successfully loaded bn2c_branch2c/gamma into the model.
Successfully loaded bn2c_branch2c/beta into the model.
Successfully loaded bn2c_branch2c/moving_mean into the model.
Successfully loaded bn2c_branch2c/moving_variance into the model.
Model restored with subset matching from: ../model/pretrained/Deeplab Resnet.ckpt
